In [0]:
!pip install ipython-autotime
%load_ext autotime

  Created wheel for ipython-autotime: filename=ipython_autotime-0.1-cp36-none-any.whl size=1832 sha256=a5d40f7e57ee8fd852b408190bfa0ace72008bd3ca115ee65e877f0f7a0ca523
  Stored in directory: /root/.cache/pip/wheels/d2/df/81/2db1e54bc91002cec40334629bc39cfa86dff540b304ebcd6e
Successfully built ipython-autotime


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import bz2
import os

time: 1.99 ms


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
time: 23.7 s


In [0]:
from google.colab import files,drive
import io
def get_labels_and_texts(file):
    labels = []
    texts = [] 
    for line in bz2.BZ2File(file):
        x = line.decode("utf-8")
        labels.append(int(x[9]) - 1)
        texts.append(x[10:].strip())
    return np.array(labels), texts

# # When First Loading Data From PC
# train_labels, train_texts = get_labels_and_texts(io.BytesIO('/content/train.ft.txt.bz2'))
# test_labels, test_texts = get_labels_and_texts(io.BytesIO('/content/test.ft.txt.bz2'))

# When loading data from Drive
train_file = open('/content/drive/My Drive/Colab Notebooks/train.ft.txt.bz2','rb')
test_file = open('/content/drive/My Drive/Colab Notebooks/test.ft.txt.bz2','rb')

train_labels, train_texts = get_labels_and_texts(io.BytesIO(train_file.read()))
test_labels, test_texts = get_labels_and_texts(io.BytesIO(test_file.read()))


time: 3min 33s


In [0]:
import re
NON_ALPHANUM = re.compile(r'[\W]')
NON_ASCII = re.compile(r'[^a-z0-1\s]')
def normalize_texts(texts):
    normalized_texts = []
    for text in texts:
        lower = text.lower()
        no_punctuation = NON_ALPHANUM.sub(r' ', lower)
        no_non_ascii = NON_ASCII.sub(r'', no_punctuation)
        normalized_texts.append(no_non_ascii)
    return normalized_texts
        
train_texts = normalize_texts(train_texts)
test_texts = normalize_texts(test_texts)

time: 2min


In [0]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

english_stop_words = stopwords.words('english')

def remove_stop_words(corpus):
    removed_stop_words = []
    for review in corpus:
        removed_stop_words.append(
            ' '.join([word for word in review.split() 
                      if word not in english_stop_words])
        )
    return removed_stop_words

train_texts = remove_stop_words(train_texts)
test_texts = remove_stop_words(test_texts)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
time: 8min 56s


First Round of Saving

In [0]:
import pickle

# Saving the objects:
with open('/content/drive/My Drive/Colab Notebooks/objs.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump([train_texts, train_labels, test_texts, test_labels], f) #, train_texts_vec, test_texts_vec], f)

time: 9.53 s


In [0]:
# Right now stop words have been successfully removed

import pickle
# Getting back the objects:
with open('/content/drive/My Drive/Colab Notebooks/objs.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
    train_texts, train_labels, test_texts, test_labels = pickle.load(f) #, train_texts_vec, test_texts_vec = pickle.load(f)

In [0]:
N

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

ngram_vectorizer = CountVectorizer(binary=False)
ngram_vectorizer.fit(train_texts)
train_texts_vec = ngram_vectorizer.transform(train_texts)
test_texts_vec = ngram_vectorizer.transform(test_texts)

time: 5min 1s


Second round of saving

In [0]:
import pickle

# Saving the objects:
with open('/content/drive/My Drive/Colab Notebooks/objs2.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump([train_texts, train_labels, test_texts, test_labels, train_texts_vec, test_texts_vec], f)

time: 35.6 s


In [0]:
# Right now text has been vectorised

import pickle
# Getting back the objects:
with open('/content/drive/My Drive/Colab Notebooks/objs2.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
    train_texts, train_labels, test_texts, test_labels, train_texts_vec, test_texts_vec = pickle.load(f)

In [0]:
RANDOM_STATE_SEED = 123
np.random.seed(RANDOM_STATE_SEED)

n_initial = 5000
n_labeled_examples = train_texts_vec.shape[0]

training_indices = np.random.randint(low=0, high = n_labeled_examples, size=n_initial)
# May not need this
X_training, y_training = train_texts_vec[training_indices], train_labels[training_indices]

time: 19 ms


K-Means on Small Subset

In [0]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=6, init='k-means++', max_iter=300, n_init=10, random_state=0)
pred_y = kmeans.fit_predict(X_training)
pred_y

array([3, 5, 5, ..., 3, 5, 3], dtype=int32)

time: 1min 16s
